# 02 – Data Integration and Enrichment

This notebook mirrors the integration logic implemented in
`scripts/integrate_data.py` and documents how the **Coffee Sales** and
**Coffee Shop** datasets are combined into a single integrated table.

Goals of this notebook:

- Load the cleaned datasets:
  - `coffee_sales_clean.csv`
  - `coffee_shop_clean.csv`
- Derive common features needed for integration (e.g., `hour_of_day`).
- Build an **hourly profile** from the coffee shop data.
- Join transaction-level sales data with the hourly profile.
- Perform basic sanity checks and summaries on the integrated dataset.
- Save the integrated table as `data/processed/coffee_integrated.csv`.

This notebook serves as documentation and an interactive complement to the script-based pipeline.

## Imports & Paths


In [1]:
import pandas as pd
from pathlib import Path

# Paths are relative to this notebook (notebooks/ folder)
PROCESSED_DIR = Path("../data/processed")
SALES_CLEAN = PROCESSED_DIR / "coffee_sales_clean.csv"
SHOP_CLEAN = PROCESSED_DIR / "coffee_shop_clean.csv"
INTEGRATED = PROCESSED_DIR / "coffee_integrated.csv"

## 1. Load cleaned datasets

We first load the cleaned versions of the Coffee Sales and Coffee Shop datasets
from `data/processed/`:

- `coffee_sales_clean.csv` – transaction-level sales data.
- `coffee_shop_clean.csv` – time-of-day–level coffee shop data.

In [2]:
sales = pd.read_csv(SALES_CLEAN)
shop = pd.read_csv(SHOP_CLEAN)

print("Sales shape:", sales.shape)
print("Shop shape: ", shop.shape)

sales.head()

Sales shape: (149116, 11)
Shop shape:  (3547, 11)


,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,1/1/23,7:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,1/1/23,7:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,1/1/23,7:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,1/1/23,7:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,1/1/23,7:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


In [3]:
shop.head()

,hour_of_day,cash_type,money,coffee_name,Time_of_Day,Weekday,Month_name,Weekdaysort,Monthsort,Date,Time
0,10,card,38.7,Latte,Morning,Fri,Mar,5,3,01/03/2024,15:50.5
1,12,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,5,3,01/03/2024,19:22.5
2,12,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,5,3,01/03/2024,20:18.1
3,13,card,28.9,Americano,Afternoon,Fri,Mar,5,3,01/03/2024,46:33.0
4,13,card,38.7,Latte,Afternoon,Fri,Mar,5,3,01/03/2024,48:14.6


## 2. Inspect columns in each dataset

Before integrating, we inspect the columns in each dataset to understand
their schemas and to inform our choice of integration keys and enrichment
features.

In [4]:
print("Sales columns:\n", sales.columns.tolist())
print("\nShop columns:\n", shop.columns.tolist())

Sales columns:
 ['transaction_id', 'transaction_date', 'transaction_time', 'transaction_qty', 'store_id', 'store_location', 'product_id', 'unit_price', 'product_category', 'product_type', 'product_detail']

Shop columns:
 ['hour_of_day', 'cash_type', 'money', 'coffee_name', 'Time_of_Day', 'Weekday', 'Month_name', 'Weekdaysort', 'Monthsort', 'Date', 'Time']


## 3. Derive `hour_of_day` and `weekday` in sales

The Coffee Shop dataset already has an `hour_of_day` column, while the
Coffee Sales dataset only has `transaction_time` and `transaction_date`.

To integrate the two datasets in a meaningful way, we:

- Parse `transaction_time` into a datetime and extract the **hour of day**.
- Parse `transaction_date` and extract the **weekday name** (optional).
- Store these derived fields as:
  - `transaction_time_parsed`
  - `hour_of_day`
  - `transaction_date_parsed`
  - `weekday`

In [5]:
# Parse transaction_time -> datetime, extract hour_of_day
sales["transaction_time_parsed"] = pd.to_datetime(
    sales["transaction_time"], errors="coerce"
)
sales["hour_of_day"] = sales["transaction_time_parsed"].dt.hour

# Parse transaction_date -> datetime, extract weekday (optional but useful)
sales["transaction_date_parsed"] = pd.to_datetime(
    sales["transaction_date"], errors="coerce"
)
sales["weekday"] = sales["transaction_date_parsed"].dt.day_name()

sales[[
    "transaction_time",
    "transaction_time_parsed",
    "hour_of_day",
    "transaction_date",
    "transaction_date_parsed",
    "weekday"
]].head()

/var/folders/kr/q1hbz4ns4tq4n3yjmwg24jsw0000gn/T/ipykernel_9006/2224531986.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales["transaction_time_parsed"] = pd.to_datetime(
/var/folders/kr/q1hbz4ns4tq4n3yjmwg24jsw0000gn/T/ipykernel_9006/2224531986.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales["transaction_date_parsed"] = pd.to_datetime(


,transaction_time,transaction_time_parsed,hour_of_day,transaction_date,transaction_date_parsed,weekday
0,7:06:11,2025-12-01 07:06:11,7,1/1/23,2023-01-01,Sunday
1,7:08:56,2025-12-01 07:08:56,7,1/1/23,2023-01-01,Sunday
2,7:14:04,2025-12-01 07:14:04,7,1/1/23,2023-01-01,Sunday
3,7:20:24,2025-12-01 07:20:24,7,1/1/23,2023-01-01,Sunday
4,7:22:41,2025-12-01 07:22:41,7,1/1/23,2023-01-01,Sunday


### 3.1 Sanity check: distribution of `hour_of_day` in sales

We examine how many transactions occur in each hour of the day. This helps
confirm that `hour_of_day` was derived correctly and reveals basic temporal
patterns in the sales data.

In [6]:
sales["hour_of_day"].value_counts().sort_index()

hour_of_day
6      4594
7     13428
8     17654
9     17764
10    18545
11     9766
12     8708
13     8714
14     8933
15     8979
16     9093
17     8745
18     7498
19     6092
20      603
Name: count, dtype: int64

## 4. Build hourly profile from Coffee Shop data

The Coffee Shop dataset contains an `hour_of_day` column along with
additional context such as:

- `money` – monetary amount associated with that hour or record,
- `Time_of_Day` – labels like Morning, Afternoon, Evening,
- `Weekday`, `Month_name`, and related ordering fields.

To create a clean time-of-day profile, we:

1. Group the coffee shop data by `hour_of_day`.
2. Compute the **average** `money` per hour.
3. Keep a representative value for selected categorical fields
   (`Time_of_Day`, `Weekday`, `Month_name`) by taking one distinct value
   per hour.

This yields a compact table with **one row per hour_of_day**.

In [7]:
# Ensure hour_of_day exists
assert "hour_of_day" in shop.columns, "hour_of_day must be present in coffee_shop_clean.csv"

# Numeric aggregation: average money per hour (if present)
agg_dict = {}
if "money" in shop.columns:
    agg_dict["money"] = "mean"   # average money per hour

# Categorical columns to keep a representative value of per hour
keep_categorical = []
for col in ["Time_of_Day", "Weekday", "Month_name"]:
    if col in shop.columns:
        keep_categorical.append(col)

# Aggregate numeric columns
grouped = shop.groupby("hour_of_day", as_index=False).agg(agg_dict)

# Attach one representative value for categorical columns
if keep_categorical:
    cat_part = (
        shop[["hour_of_day"] + keep_categorical]
        .drop_duplicates(subset=["hour_of_day"])
    )
    shop_hour_profile = grouped.merge(cat_part, on="hour_of_day", how="left")
else:
    shop_hour_profile = grouped

shop_hour_profile.sort_values("hour_of_day").head(10)

,hour_of_day,money,Time_of_Day,Weekday,Month_name
0,6,29.880000,Morning,Mon,Feb
1,7,32.341136,Morning,Sun,May
2,8,29.863319,Morning,Tue,May
3,9,30.017686,Morning,Tue,Mar
4,10,31.093049,Morning,Fri,Mar
5,11,29.869611,Morning,Sat,Mar
6,12,30.786805,Afternoon,Fri,Mar
7,13,31.238933,Afternoon,Fri,Mar
8,14,31.883556,Afternoon,Sat,Mar
9,15,31.678051,Afternoon,Fri,Mar


In [8]:
shop_hour_profile.describe(include="all")

,hour_of_day,money,Time_of_Day,Weekday,Month_name
count,17.000000,17.000000,17,17,17
unique,NaN,NaN,3,6,4
top,NaN,NaN,Morning,Fri,Mar
freq,NaN,NaN,6,7,11
mean,14.000000,31.656390,NaN,NaN,NaN
std,5.049752,1.245912,NaN,NaN,NaN
min,6.000000,29.863319,NaN,NaN,NaN
25%,10.000000,30.786805,NaN,NaN,NaN
50%,14.000000,31.883556,NaN,NaN,NaN
75%,18.000000,32.488633,NaN,NaN,NaN


## 5. Merge sales data with hourly profile

We now integrate the two datasets by joining transaction-level sales with
the hourly coffee shop profile on `hour_of_day`.

Conceptually:

- Each row in `sales` is a transaction with a derived `hour_of_day`.
- Each row in `shop_hour_profile` summarizes average `money` and typical
  time-of-day labels for that hour.
- A left join from sales to the hourly profile enriches each transaction
  with additional **time-of-day context**.

In [9]:
integrated = sales.merge(
    shop_hour_profile,
    on="hour_of_day",
    how="left",
    suffixes=("_sale", "_shop"),
)

integrated.shape

(149116, 19)

In [10]:
integrated.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,transaction_time_parsed,hour_of_day,transaction_date_parsed,weekday,money,Time_of_Day,Weekday,Month_name
0,1,1/1/23,7:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg,2025-12-01 07:06:11,7,2023-01-01,Sunday,32.341136,Morning,Sun,May
1,2,1/1/23,7:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,2025-12-01 07:08:56,7,2023-01-01,Sunday,32.341136,Morning,Sun,May
2,3,1/1/23,7:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,2025-12-01 07:14:04,7,2023-01-01,Sunday,32.341136,Morning,Sun,May
3,4,1/1/23,7:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm,2025-12-01 07:20:24,7,2023-01-01,Sunday,32.341136,Morning,Sun,May
4,5,1/1/23,7:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,2025-12-01 07:22:41,7,2023-01-01,Sunday,32.341136,Morning,Sun,May


### 5.1 Sanity checks and basic summaries

To validate the integrated dataset, we:

- Inspect the list of columns.
- Compute summary statistics by `hour_of_day`, such as:
  - Average revenue (if we define `revenue = transaction_qty * unit_price`).
  - Number of transactions per hour.
- Examine how transactions are distributed across `Time_of_Day` categories
  and across combinations of `store_location` and `Time_of_Day`.

These checks both verify that the join worked as expected and provide
early insights for the analysis section of the project.

In [11]:
integrated.columns.tolist()

['transaction_id',
 'transaction_date',
 'transaction_time',
 'transaction_qty',
 'store_id',
 'store_location',
 'product_id',
 'unit_price',
 'product_category',
 'product_type',
 'product_detail',
 'transaction_time_parsed',
 'hour_of_day',
 'transaction_date_parsed',
 'weekday',
 'money',
 'Time_of_Day',
 'Weekday',
 'Month_name']

In [12]:
# 1) Average revenue (qty * unit_price) by hour_of_day
if {"transaction_qty", "unit_price"}.issubset(integrated.columns):
    integrated["revenue"] = integrated["transaction_qty"] * integrated["unit_price"]
    avg_rev_by_hour = integrated.groupby("hour_of_day")["revenue"].mean().round(2)
    avg_rev_by_hour

In [13]:
# 2) Number of transactions by hour_of_day
txn_count_by_hour = integrated["hour_of_day"].value_counts().sort_index()
txn_count_by_hour

hour_of_day
6      4594
7     13428
8     17654
9     17764
10    18545
11     9766
12     8708
13     8714
14     8933
15     8979
16     9093
17     8745
18     7498
19     6092
20      603
Name: count, dtype: int64

In [14]:
# 4) Example: transactions by store_location and Time_of_Day
if "store_location" in integrated.columns and "Time_of_Day" in integrated.columns:
    txn_by_loc_time = (
        integrated.groupby(["store_location", "Time_of_Day"])["transaction_id"]
        .count()
        .reset_index(name="num_transactions")
        .sort_values("num_transactions", ascending=False)
    )
    txn_by_loc_time.head(10)

## 6. Save integrated dataset

Finally, we save the integrated table to `data/processed/coffee_integrated.csv`
so it can be reused by downstream notebooks (e.g., `03_eda.ipynb`) and by
the automated pipeline.

Note: this notebook uses the same filename as the script
`scripts/integrate_data.py`, so either approach (script or notebook) will
produce a compatible integrated dataset.

In [15]:
INTEGRATED = PROCESSED_DIR / "coffee_integrated.csv"
integrated.to_csv(INTEGRATED, index=False)
INTEGRATED

PosixPath('../data/processed/coffee_integrated.csv')